In [1]:
#!pip install segmentation_models_pytorch   # Install dependencies
#!pip install timm
#!pip install -U git+https://github.com/albu/albumentations --no-cache-dir
#!pip install -U git+https://github.com/albumentations-team/albumentations
#!pip install albumentations
#!pip install segmentation_models_pytorch
import segmentation_models_pytorch as smp
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import albumentations as albu
import torch
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [22]:
DATA_DIR = './'
x_test_dir = 'test1/image'      # Note: put the dataset on the current path.
y_test_dir = 'test1/label/'  # You can modify it to test on the 2-80 Additional Original Images

In [23]:
DEVICE = 'cuda'
CLASSES = ['crack']
ENCODER = 'se_resnext50_32x4d'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = ['crack']
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multicalss segmentation
# create segmentation model with pretrained encoder
preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)
fpn = torch.load('fpn.pth',map_location=DEVICE).to(DEVICE)

D:\Users\PC\Anaconda3\envs\pytorch2\lib\site-packages\torch\serialization.py:656: SourceChangeWarning: source code of class 'segmentation_models_pytorch.fpn.model.FPN' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
D:\Users\PC\Anaconda3\envs\pytorch2\lib\site-packages\torch\serialization.py:656: SourceChangeWarning: source code of class 'segmentation_models_pytorch.encoders.senet.SENetEncoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
D:\Users\PC\Anaconda3\envs\pytorch2\lib\site-packages\torch\serialization.py:656: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can ret

In [4]:
# from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
# get the imread result from test set((3864, 5152, 3)or(3264, 4928, 3) numpy), return numpy of  （98，3，480，640）or（145，3，480，640）
def testdataprocessing(img):
    h,w,c = img.shape
    #padding the images with zeros
    if h == 2448:
        img = cv2.copyMakeBorder(img , 216, 216, 288 , 288, cv2.BORDER_CONSTANT, value=0)
        ##(6*480=2880,6*640=3840)
    elif h == 3024:
        img = cv2.copyMakeBorder(img,168,168,224 , 224, cv2.BORDER_CONSTANT, value=0)
    output = np.zeros([1,480,640,3], dtype=int)
     #(7*480=3360,7*640=4480)
    h,w,c = img.shape
    #set the stride in colomns/rows is 480/640
    stride_h = 480
    stride_w = 640
    if h == 2880:
        column = int(h/480)
        row = int(w/640)
        for i in range(row):
            for j in range(column):
                term = np.zeros([480,640,3], dtype=int)
                term = img[j*stride_h:j*stride_h + stride_h, i*stride_w : i*stride_w+stride_w,:].copy() 
                term = term[np.newaxis,:,:,:]
                output = np.concatenate((output,term),axis = 0)
        output = np.delete(output,0,axis = 0)
        #get the center part of raw images and split again
        img_seg = img[240:2640,320:3520,:].copy()
        for i in range(row-1):
            for j in range(column-1):
                term = np.zeros([480,640,3], dtype=int)
                term = img_seg[j*stride_h:j*stride_h + stride_h, i*stride_w : i*stride_w+stride_w,:].copy() 
                term = term[np.newaxis,:,:,:]
                output = np.concatenate((output,term),axis = 0)

    if h == 3360:
        column = int(h/480)
        row = int(w/640)
        for i in range(row):
            for j in range(column):
                term = np.zeros([480,640,3], dtype=int)
                term = img[j*stride_h:j*stride_h + stride_h, i*stride_w : i*stride_w+stride_w,:].copy() 
                term = term[np.newaxis,:,:,:]
                output = np.concatenate((output,term),axis = 0)
        output = np.delete(output,0,axis = 0)
        #get the center part of raw images and split again
        img_seg = img[240:3120,320:4160,:].copy()
        for i in range(row-1):
            for j in range(column-1):
                term = np.zeros([480,640,3], dtype=int)
                term = img_seg[j*stride_h:j*stride_h + stride_h, i*stride_w : i*stride_w+stride_w,:].copy() 
                term = term[np.newaxis,:,:,:]
                output = np.concatenate((output,term),axis = 0)
    return output

#trainsofrm the numpy of （98，480，640）/（145，480，640）back into (3864，5152）or（3264，4928）
def visualizaion(output):
    n,stride_h,stride_w = output.shape    
    if n == 61:
        count = 0
        #combine the firt part of predition
        mask_1 = np.zeros([2880,3840], dtype=int)
        for i in range (6):
            for j in range(6):
                mask_1[j*stride_h:j*stride_h + stride_h, i*stride_w : i*stride_w+stride_w] = output[count,:,:].copy()  
                count += 1
        mask_2 = np.zeros([2400,3200], dtype=int)     
        #combine the second part of prediction
        for i in range (5):
            for j in range(5):
                mask_2[j*stride_h:j*stride_h + stride_h, i*stride_w : i*stride_w+stride_w] = output[count,:,:].copy()  
                count += 1     
        #sum the mask1 and mask2 in pixel level
        mask_3 = mask_1[240:2640,320:3520].copy() + mask_2[:,:].copy()
        mask_3 = mask_3*0.5
        prediction = mask_1.copy()
        prediction[240:2640,320:3520] = mask_3[:,:]
        p = prediction[216:2664,288:3552].copy()    
    if n == 85:
        count = 0
        #combine the first part of prediction
        mask_1 = np.zeros([3360,4480], dtype=int)
        for i in range (7):
            for j in range(7):
                mask_1[j*stride_h:j*stride_h + stride_h, i*stride_w : i*stride_w+stride_w] = output[count,:,:].copy()  
                count += 1
        mask_2 = np.zeros([2880,3840], dtype=int) 
        #combine the second part of prediction
        for i in range (6):
            for j in range(6):
                mask_2[j*stride_h:j*stride_h + stride_h, i*stride_w : i*stride_w+stride_w] = output[count,:,:].copy()  
                count += 1 
        #sum the mask1 and mask2 in pixel level
        mask_3 = mask_1[240:3120,320:4160].copy() + mask_2[:,:].copy()
        mask_3 = mask_3*0.5
        prediction = mask_1.copy()
        prediction[240:3120,320:4160] = mask_3[:,:]
        p = prediction[168:3192,224:4256].copy()        
    return p

In [24]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
# get the imread result from test set((3864, 5152, 3)or(3264, 4928, 3) numpy), return numpy of  （98，3，480，640）or（145，3，480，640）
def testdataprocessing(img):
    output = np.zeros([1,320,320,3], dtype=int)
    term = img[np.newaxis,:,:,:]
    output = np.concatenate((output,term),axis = 0)
    print(output.shape)
    return output


In [25]:
def visualizaion(output):
    n,stride_h,stride_w = output.shape    
    
    mask = np.zeros([320,320], dtype=int)
    mask[:,:] = output[1,:,:].copy()         
    return mask

In [26]:
# Define the dataset
class Dataset(BaseDataset):
    """
    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    Note: we don't need mask when testing.
    
    """
    
    CLASSES = ['crack']
    
    def __init__(
            self, 
            images_dir, 
            classes=None, 
            augmentation=None, 
            preprocessing=None,
    ):
        self.ids = sorted(os.listdir(images_dir))
        #将image文件夹下的所有照片名称创建为一个列表并且顺序排列['001.png','002.png'.....]
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        #将所有照片路径创建为列表['test1/image\\0001.png','test1/image\\0002.png'...]
        
        # convert str names to class values on masks名称变成值？
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        
        # read data
        image = cv2.imread(self.images_fps[i],1)
        #1加载彩色，0加载灰度，-1还读入alpha通道代表透明度
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #opencv默认通道是bgr，这是早期留下来的问题。因此将bgr转换为rgb
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image)
            image = sample['image']
        if self.preprocessing:
            sample = self.preprocessing(image=image)
            image = sample['image']
        image = testdataprocessing(image.transpose(1,2,0))
        image = image.transpose(0,3,1,2)
        return image
        
    def __len__(self):
        return len(self.ids)
    #暂时不知道什么用

In [27]:
# create test dataset
def get_validation_augmentation():
    """You can put some augmentations on here"""
    #可以加入不同的操作，只需放在test_transform列表里。Resize可以不改变照片内容，改变图片像素尺寸
    test_transform = [
     albu.Resize(320,320),
    ]
    return albu.Compose(test_transform)
    #就是将所有操作打包起来，对对象操作时：image3=albu.Compose(test_transform)(image=image_name)['image']
def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')
def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor),
    ]
    return albu.Compose(_transform)
    
test_dataset = Dataset(
    x_test_dir, 
    augmentation=None, # for test, we just set augmentation to None
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=CLASSES,
)


In [28]:
import matplotlib
pre_1 = []
os.system('mkdir Demo')
# output mask
for n in range(len(test_dataset)):
    print(n)
    image = test_dataset[n]
    x_tensor = torch.FloatTensor(image).to(DEVICE)
    shape = x_tensor.shape[0]//3    # reduce memory on GPU
    pr_mask = []
    with torch.no_grad():
        mask = fpn.predict(x_tensor).squeeze().detach().cpu().numpy().round()
        pr_mask.append(mask)
#         mask = fpn.predict(x_tensor[shape:shape*2]).squeeze().detach().cpu().numpy().round()
#         pr_mask.append(mask)
#         mask = fpn.predict(x_tensor[shape*2:]).squeeze().detach().cpu().numpy().round()
#         pr_mask.append(mask)
    pr_mask = np.concatenate(np.array(pr_mask),axis=0)  # concatenate three pieces
    pr_mask= visualizaion(pr_mask)
    pr_mask[pr_mask > 0]=1
    pre_1.append(pr_mask)
    matplotlib.image.imsave("Demo2/%04.d.png"%(n+1), pr_mask)


0
(2, 320, 320, 3)
1
(2, 320, 320, 3)
2
(2, 320, 320, 3)
3
(2, 320, 320, 3)
4
(2, 320, 320, 3)


In [10]:
# calculate iou
data = []
for j,i in enumerate(sorted(os.listdir(x_test_dir))):
    mask_1 = cv2.imread("Demo/"+i,0)
    mask_1[mask_1==30]=0
    mask_1[mask_1==215]=1
    mask_2 = cv2.imread(y_test_dir+i, 0)
    mask_2[mask_2>0]=1
    mask_3 = mask_1 & mask_2
    mask_2[mask_1>0]=1
    iou = mask_3.sum()/mask_2.sum()
    print(str(j+1),':',"%.4f" % (iou))
    data.append(iou)
print("Final miou:", np.mean(data))

1 : 0.5852
2 : 0.5167
3 : 0.5706
4 : 0.6068
5 : 0.6344
6 : 0.8475
7 : 0.8503
8 : 0.8307
9 : 0.8206
10 : 0.8184
11 : 0.7651
12 : 0.8063
13 : 0.7736
14 : 0.6648
15 : 0.8392
16 : 0.8998
17 : 0.8346
18 : 0.7713
19 : 0.8126
20 : 0.8326
Final miou: 0.7540582825979366


In [11]:
from torchsummary import summary

In [12]:
summary(fpn,(3,320, 320))

TypeError: 'int' object is not callable

FPN(
  (encoder): SENetEncoder(
    (layer0): Sequential(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (layer1): Sequential(
      (0): SEResNeXtBottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu)